In [2]:
from flask import Flask, render_template, request, redirect, url_for
import os
import pandas as pd

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Create the upload folder if it doesn't exist
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

class DataIngestion:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None
        self.file_type = self.infer_file_type()

    def infer_file_type(self):
        _, file_extension = os.path.splitext(self.file_path)
        return file_extension.lower()

    def ingest_file(self):
        if self.file_type == ".csv":
            self.data = self.read_csv()
        elif self.file_type in [".xls", ".xlsx"]:
            self.data = self.read_excel()
        elif self.file_type == ".json":
            self.data = self.read_json()
        elif self.file_type == ".parquet":
            self.data = self.read_parquet()
        elif self.file_type == ".yaml" or self.file_type == ".yml":
            self.data = self.read_yaml()
        else:
            raise ValueError(f"Unsupported file type: {self.file_type}")

    def read_csv(self):
        return pd.read_csv(self.file_path)

    def read_excel(self):
        return pd.read_excel(self.file_path)

    def read_json(self):
        with open(self.file_path, 'r') as f:
            return json.load(f)

    def read_parquet(self):
        return pd.read_parquet(self.file_path)

    def read_yaml(self):
        with open(self.file_path, 'r') as f:
            return yaml.safe_load(f)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return "No file part", 400

    file = request.files['file']
    if file.filename == '':
        return "No selected file", 400

    file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(file_path)

    # Ingest the uploaded file
    data_ingestion = DataIngestion(file_path)
    data_ingestion.ingest_file()

    # Optionally return some kind of response
    return "File uploaded and ingested successfully"

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\zirot\anaconda3\envs\Test\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
